In [ ]:
# integrating data from a NoSQL database - reviews in product dimension - denormalization



1. I use the 'faker' python module to generate some random review data for each product
2. Then I get these reviews from the MongoDB,
3. Transform reviews into a DataFrame,
4. Add columns 'reviewer', 'review_text', and 'review_rating' to the 'product_dim' table in my SQL database,
5. Update the 'product_dim' with the new review data.



In [ ]:
import pymongo
import pandas as pd
import pymysql
from faker import Faker

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["local"]

In [ ]:
reviews_collection = db["product_reviews"]
fake = Faker()

for i in range(110): # generates 110 random reviews in our mongoDB collection
    review = {
        "product_key": i+1,
        "reviewer": fake.name(),
        "review_text": fake.paragraph(nb_sentences=3),
        "rating": fake.random_int(min=1, max=5)
    }
    reviews_collection.insert_one(review)

In [ ]:
results = reviews_collection.find()

In [ ]:
reviews_df = pd.DataFrame(list(results))

In [ ]:
reviews_df.head(5)

,_id,product_key,reviewer,review_text,rating
0,660de075fac3316f28118801,1,Jennifer Lawson,Forget second police admit beautiful item. Pro...,2
1,660de075fac3316f28118802,2,Ian Melendez,Interesting senior someone half. Church throug...,4
2,660de075fac3316f28118803,3,Travis Jackson,Agency skin himself option paper subject upon....,2
3,660de075fac3316f28118804,4,Scott Shaw,Quality fall much crime could. Today network p...,3
4,660de075fac3316f28118805,5,Abigail Ross,Know cultural be tell local. Career suddenly h...,3


Now we can add in our mongoDB data to our sql table:

In [ ]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Passw0rd123',
    'database': 'retail_sales_dw'
}
conn = pymysql.connect(**db_config)
product_dim = pd.read_sql('SELECT product_key, product_id, product_name, product_line, quantity_in_stock, product_price FROM product_dim', conn)
merged_df = pd.merge(product_dim, reviews_df, on=['product_key'], how='left')
cursor = conn.cursor()

C:\Users\ds2002-student\AppData\Local\Temp\2\ipykernel_9040\2646952924.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product_dim = pd.read_sql('SELECT product_key, product_id, product_name, product_line, quantity_in_stock, product_price FROM product_dim', conn)


In [ ]:
cursor.execute("ALTER TABLE product_dim ADD COLUMN reviewer varchar(50)")
cursor.execute("ALTER TABLE product_dim ADD COLUMN review_text text")
cursor.execute("ALTER TABLE product_dim ADD COLUMN rating INT")

0

In [ ]:
for index, row in merged_df.iterrows():
    update_query = """
        UPDATE product_dim
        SET reviewer = %s
        WHERE product_key = %s
    """
    values = (row['reviewer'], row['product_key'])
    cursor.execute(update_query, values)

In [ ]:
for index, row in merged_df.iterrows():
    update_query = """
        UPDATE product_dim
        SET review_text = %s
        WHERE product_key = %s
    """
    values = (row['review_text'], row['product_key'])
    cursor.execute(update_query, values)

In [ ]:
for index, row in merged_df.iterrows():
    update_query = """
        UPDATE product_dim
        SET rating = %s
        WHERE product_key = %s
    """
    values = (row['rating'], row['product_key'])
    cursor.execute(update_query, values)

In [ ]:
conn.commit()
conn.close()